In [1]:
from dataset.semantic_spray_dataset import SemanticSprayDataset
from visualization import visualization_tools as vis
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from preprocess import distance_point_filter
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.linear_model import RANSACRegressor

In [2]:
train_dataset = SemanticSprayDataset("SemanticSprayDataset", "train")
test_dataset = SemanticSprayDataset("SemanticSprayDataset", "train")

Mode: train, loaded samples 8389
Mode: train, loaded samples 8389


In [3]:
def dbscan(data, eps=2, min_samples=5, leaf_size=30):
    clustering = DBSCAN(eps=eps, min_samples=min_samples, leaf_size=leaf_size).fit(data)

    core_samples_mask = np.zeros_like(clustering.labels_, dtype=bool)
    core_samples_mask[clustering.core_sample_indices_] = True
    labels = clustering.labels_
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)
    print('Estimated number of clusters: %d' % n_clusters_)
    print('Estimated number of noise points: %d' % n_noise_)

    unique_labels = set(labels)
    labels_color = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]
    return labels, labels_color

def points_preprocess(data, distance_crop=False):
    boundaries = {}
    boundaries['min_x'] = -30
    boundaries['max_x'] = 30
    boundaries['min_y'] = -30
    boundaries['max_y'] = 30
    boundaries['min_z'] = -30
    boundaries['max_z'] = 30
    if distance_crop:
        data["points"], data["labels"] = distance_point_filter(data["points"], boundaries, data["labels"])

    return data

In [4]:
data = next(iter(test_dataset))

In [5]:
data["labels"] = np.zeros(len(data["labels"]))
data = points_preprocess(data, distance_crop=True)
labels, labels_color = dbscan(data["points"], 2, 20)

data["labels"] = labels

vis.visualize_scene(data, plot_type="3D", labels_color=labels_color)

Estimated number of clusters: 64
Estimated number of noise points: 6270


Widget(value="<iframe src='http://localhost:43457/index.html?ui=P_0x7f63602353d0_0&reconnect=auto' style='widt…

In [6]:
import pyransac3d as pyrsc

In [9]:
data = next(iter(test_dataset))

In [10]:
data = points_preprocess(data, distance_crop=True)
points = data['points'][:,:3]
labels = data['labels']
points.shape

(19232, 3)

In [11]:
vis.visualize_scene(data, plot_type="3D")

Widget(value="<iframe src='http://localhost:43457/index.html?ui=P_0x7f63381d3520_2&reconnect=auto' style='widt…

In [12]:
help(pyrsc.Plane.fit)

Help on function fit in module pyransac3d.plane:

fit(self, pts, thresh=0.05, minPoints=100, maxIteration=1000)
    Find the best equation for a plane.
    
    :param pts: 3D point cloud as a `np.array (N,3)`.
    :param thresh: Threshold distance from the plane which is considered inlier.
    :param maxIteration: Number of maximum iteration which RANSAC will loop over.
    :returns:
    - `self.equation`:  Parameters of the plane using Ax+By+Cy+D `np.array (1, 4)`
    - `self.inliers`: points from the dataset considered inliers
    
    ---



In [13]:
plane1 = pyrsc.Plane()
best_eq, best_inliers = plane1.fit(points, 0.1, 1000, 1000)
print(best_eq)
best_inliers.shape

[-0.0032216029, -0.0023162742, -0.99999213, 1.7271638]


(11717,)

In [14]:
outliners_points = points[best_inliers]
outliners_labels = labels[best_inliers]
outliners_data = {'points': outliners_points, 'labels': outliners_labels}

cleaned_points = np.delete(points, best_inliers, 0)
cleaned_labels = np.delete(labels, best_inliers, 0)
cleaned_data = {'points': cleaned_points, 'labels': cleaned_labels}
cleaned_data['points'].shape

(7515, 3)

In [15]:
vis.visualize_scene(outliners_data, plot_type="3D")

Widget(value="<iframe src='http://localhost:43457/index.html?ui=P_0x7f63381c4610_3&reconnect=auto' style='widt…

In [16]:
vis.visualize_scene(cleaned_data, plot_type="3D")

Widget(value="<iframe src='http://localhost:43457/index.html?ui=P_0x7f633824d580_4&reconnect=auto' style='widt…

In [17]:
labels, labels_color = dbscan(cleaned_data["points"], 2, 20)

cleaned_data["labels"] = labels

vis.visualize_scene(cleaned_data, plot_type="3D", labels_color=labels_color)

Estimated number of clusters: 13
Estimated number of noise points: 246


Widget(value="<iframe src='http://localhost:43457/index.html?ui=P_0x7f6338254340_5&reconnect=auto' style='widt…